In [1]:
# load libraries

import os
import numpy as np
from statistics import mean

import torch
import torchvision

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from scipy import stats

from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

PROJECT_PATH = os.getenv('HOME') + '/aiffel/0_felton'
MODEL_PATH = os.path.join(PROJECT_PATH, 'weights')
DATA_PATH = os.path.join(PROJECT_PATH, 'data_ex_rear2')
TRAIN_PATH = os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
REJECT_PATH = os.path.join(DATA_PATH, 'reject')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # connected to GPU if 'cuda' is printed here

cuda


In [ ]:
# checking imgs in a folder

for dirpath, dirnames, filenames in os.walk(TRAIN_PATH):
    for i, filename in enumerate(filenames):
        print(os.path.join(dirpath, filename)) # prints file names
        image = Image.open(os.path.join(dirpath, filename), 'r')
        print(f'size: ({image.width}, {image.height}, {image.getbands()})') # prints img info
        plt.imshow(image)
        plt.show()
        break # print 1 per folder

In [4]:
# Normalize imgs, resize to 224x224
# Create pipeline
# PyTorch offers various augmentation techniques in torchvision.transforms.Compose

def create_dataloader(path, batch_size, istrain):
    nearest_mode = torchvision.transforms.InterpolationMode.NEAREST
    normalize = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
    train_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.ColorJitter(),
        torchvision.transforms.ToTensor(),
        normalize
    ])

    test_transformer = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224), interpolation=nearest_mode),
        torchvision.transforms.ToTensor(),
        normalize
    ])
    
    if istrain:
        data = torchvision.datasets.ImageFolder(path, transform=train_transformer)
        dataloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
        
    else:
        data = torchvision.datasets.ImageFolder(path, transform=test_transformer)
        dataloader = torch.utils.data.DataLoader(data, shuffle=False)

    return dataloader, data

In [5]:
# creating train dataset

BATCH_SIZE = 32

train_loader, _train_data = create_dataloader(TRAIN_PATH, BATCH_SIZE, True)
target_class_num = len(os.listdir(os.path.join(TRAIN_PATH)))

print('target_class_num: ', target_class_num)
print('train: ', _train_data.class_to_idx)

target_class_num:  2
train:  {'12_inner_rear_seat_train_2000': 0, 'resized_14_inner_sheet_dirt_train': 1}


In [6]:
# checking num of imgs in each class

for rootpath, dirpath, filenames in os.walk(TRAIN_PATH):
    print(f'{rootpath} : {len(filenames)}')

/aiffel/aiffel/0_felton/data_ex_rear2/train : 0
/aiffel/aiffel/0_felton/data_ex_rear2/train/12_inner_rear_seat_train_2000 : 2000
/aiffel/aiffel/0_felton/data_ex_rear2/train/resized_14_inner_sheet_dirt_train : 2000


In [7]:
# creating test dataset
# shuffle = False
test_loader, _test_data = create_dataloader(TEST_PATH, BATCH_SIZE, False)
print('test: ', _test_data.class_to_idx)

test:  {'12_inner_rear_seat_test_500': 0, 'resized_14_inner_sheet_dirt_test': 1}


In [8]:
# checking num of imgs in each class

for rootpath, dirpath, filenames in os.walk(TEST_PATH):
    print(f'{rootpath} : {len(filenames)}')

/aiffel/aiffel/0_felton/data_ex_rear2/test : 0
/aiffel/aiffel/0_felton/data_ex_rear2/test/resized_14_inner_sheet_dirt_test : 500
/aiffel/aiffel/0_felton/data_ex_rear2/test/12_inner_rear_seat_test_500 : 500


### model 훈련

In [9]:
# metrics from sklearn.metrics

def calculate_metrics(trues, preds):
    accuracy = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average='macro')
    precision = precision_score(trues, preds, average='macro')
    recall = recall_score(trues, preds, average='macro')
    return accuracy, f1, precision, recall

In [10]:
# train function

def train(dataloader, net, learning_rate, weight_decay_level, device):
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        net.parameters(),
        lr = learning_rate, 
        weight_decay = weight_decay_level
    )

    net.train()

    train_losses = list()
    train_preds = list()
    train_trues = list()

    for idx, (img, label) in enumerate(dataloader):

        img = img.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()

        out = net(img)

        _, pred = torch.max(out, 1)
        loss = criterion(out, label)

        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_trues.extend(label.view(-1).cpu().numpy().tolist())
        train_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(train_trues, train_preds)

    print('\n''====== Training Metrics ======')
    print('Loss: ', mean(train_losses))
    print('Acc: ', acc)
    print('F1: ', f1)
    print('Precision: ', prec)
    print('Recall: ', rec)
    print(confusion_matrix(train_trues, train_preds))

    return net, acc, f1, prec, rec

In [11]:
# test function

def test(dataloader, net, device):

    criterion = torch.nn.CrossEntropyLoss()
    
    net.eval()
    test_losses = list()
    test_trues = list()
    test_preds = list()
    
    with torch.no_grad():
        for idx, (img, label) in enumerate(dataloader):

            img = img.to(device)
            label = label.to(device)

            out = net(img)

            _, pred = torch.max(out, 1)
            loss = criterion(out, label)

            test_losses.append(loss.item())
            test_trues.extend(label.view(-1).cpu().numpy().tolist())
            test_preds.extend(pred.view(-1).cpu().detach().numpy().tolist())

    acc, f1, prec, rec = calculate_metrics(test_trues, test_preds)

    print('====== Test Metrics ======')
    print('Test Loss: ', mean(test_losses))
    print('Test Acc: ', acc)
    print('Test F1: ', f1)
    print('Test Precision: ', prec)
    print('Test Recall: ', rec)
    print(confusion_matrix(test_trues, test_preds))

    return net, acc, f1, prec, rec

In [12]:
# code to save best params based on acc

def train_classifier(net, train_loader, test_loader, n_epochs, learning_rate, weight_decay, device):
    best_test_acc = 0
    
    model_save_path = None
    model_save_base = 'weights'
    if not os.path.exists(model_save_base):
        os.makedirs(model_save_base)
    
    print('>> Start Training Model!')
    for epoch in range(n_epochs):
        
        print('> epoch: ', epoch)

        net, _, _, _, _ = train(train_loader, net, learning_rate, weight_decay, device)
        net, test_acc, _, _, _  = test(test_loader, net, device)

        if test_acc > best_test_acc:

            best_test_acc = test_acc
            test_acc_str = '%.5f' % test_acc

            print('[Notification] Best Model Updated!')
            model_save_path = os.path.join(model_save_base, 'classifier_acc_' + str(test_acc_str) + '.pth') 
            torch.save(net.state_dict(), model_save_path)
                
    return model_save_path

In [13]:
# load pre-trained resnet50

net = torchvision.models.resnet50(pretrained=True)
net.fc = torch.nn.Linear(
    net.fc.in_features,
    target_class_num
)

net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### 모델 학습 시키기

In [14]:
# test with 1 epoch

EPOCHS = 1
LEARNING_RATE = 0.005
WEIGHT_DECAY = 0.0005

saved_weight_path = train_classifier(net, train_loader, test_loader, EPOCHS, LEARNING_RATE, WEIGHT_DECAY, device)

>> Start Training Model!
> epoch:  0

====== Training Metrics ======
Loss:  0.3388406404554844
Acc:  0.88075
F1:  0.8805564940551502
Precision:  0.8832334485549985
Recall:  0.88075
[[1681  319]
 [ 158 1842]]
====== Test Metrics ======
Test Loss:  0.15834427163157064
Test Acc:  0.938
Test F1:  0.9377907280090223
Test Precision:  0.943974115700871
Test Recall:  0.938
[[440  60]
 [  2 498]]
[Notification] Best Model Updated!


In [15]:
# result

EPOCHS = 20
LEARNING_RATE = 0.005
WEIGHT_DECAY = 0.0005

saved_weight_path = train_classifier(net, train_loader, test_loader, EPOCHS, LEARNING_RATE, WEIGHT_DECAY, device)

>> Start Training Model!
> epoch:  0

====== Training Metrics ======
Loss:  0.20420244591683148
Acc:  0.9455
F1:  0.945493404701969
Precision:  0.9457157264115832
Recall:  0.9455
[[1869  131]
 [  87 1913]]
====== Test Metrics ======
Test Loss:  0.08185703790359003
Test Acc:  0.977
Test F1:  0.9769898525249636
Test Precision:  0.977842914901887
Test Recall:  0.977
[[478  22]
 [  1 499]]
[Notification] Best Model Updated!
> epoch:  1

====== Training Metrics ======
Loss:  0.09436291536688804
Acc:  0.976
F1:  0.9759997059963985
Precision:  0.976023325142932
Recall:  0.976
[[1945   55]
 [  41 1959]]
====== Test Metrics ======
Test Loss:  0.7899331645962083
Test Acc:  0.598
Test F1:  0.5214285714285715
Test Precision:  0.7722222222222221
Test Recall:  0.598
[[ 99 401]
 [  1 499]]
> epoch:  2

====== Training Metrics ======
Loss:  0.06037014590762556
Acc:  0.983
F1:  0.982999893749336
Precision:  0.9830120753018825
Recall:  0.9830000000000001
[[1961   39]
 [  29 1971]]
====== Test Metrics ==